<h1>NLP_PROBLEM</h1>
<h1>___________________________________</h1>

<h2>Getting the data</h2>

In [2]:
import pandas as pd # It will help to read the csv files and create the dataframe

Reading the training and testing data from file

In [3]:
data = pd.read_csv('V1.4_Training.csv')
data_test = pd.read_csv('SubtaskA_Trial_Test.csv')

In [4]:
data.head()

,ID,COMMENT,LABEL
0,663_3,"""Please enable removing language code from the...",1
1,663_4,"""Note: in your .csproj file, there is a Suppor...",0
2,664_1,"""Wich means the new version not fully replaced...",0
3,664_2,"""Some of my users will still receive the old x...",0
4,664_3,"""The store randomly gives the old xap or the n...",0


In [5]:
data_test.head()

,id,comment,label
0,13101,"""I'm not asking Microsoft to Gives permission ...",X
1,13121,"""somewhere between Android and iPhone.""",X
2,13131,"""And in the Windows Store you can flag the App...",X
3,13132,"""Many thanks Sameh Hi, As we know, there is a ...",X
4,13133,"""The idea is that we can develop a regular app...",X


Saving all the rows and the first index(TEXT DATA)

In [6]:
df1 = data.iloc[:, 1]

Appending all the training text data to a list

In [7]:
train_x = []
for line in df1:
    train_x.append(line)

Getting all the rows and second index(LABELS) and saving them to a list

In [8]:
df2 = data.iloc[:, 2]
train_y = []
for line in df2:
    train_y.append(line)

Similarly for the test data

In [9]:
test_x = []
df3 = data_test.iloc[:, 1]
for line in df3:
    test_x.append(line)

The given "SubtaskA_Trial_Test_Labeled.csv" has some errors so, copied it the column containing the labels and pasted it to a test file then it is read through all labels and saved it to a list as int



In [10]:
with open('test_label.txt', 'r') as f:
    test_y_file = f.readlines()

test_y = []
for val in test_y_file:
    test_y.append(val[0])
for index, val in enumerate(test_y):
    val = int(val)
    test_y[index] = val
    

Printing the length of training and testing data

In [11]:
print("Train-set size: ", len(train_x))
print("Test-set size: ", len(test_x))

Train-set size:  8500
Test-set size:  592


In [12]:
index = 6
print(train_x[index])
print(train_y[index])

"The wp7 xap works only on WP7 and the wp8 xap works only for WP8."
0


<h2>Text Preprocessing</h2>

The text data contains uppercase letters, numbers and punctuations. Having these makes the vocabulary go high unnecessarily which is bad for our model. So here the text data is <b>lowered</b>, <b>removed numbers</b>, <b>removed punctuations</b>.

In [13]:
import re 
import string

In [14]:
for index ,line in enumerate(train_x):
    line = line.lower() #lowercase lines
    line = re.sub(r'\d+', '', line) #regular expression to remove the numbers
    translation = str.maketrans(" "," ", string.punctuation)
    line = line.translate(string.punctuation) #removes the punctuations
    line = line.translate(translation)
    train_x[index] = line # update the text files
    
for index ,line in enumerate(test_x):
    line = line.lower()
    line = re.sub(r'\d+', '', line)
    translation = str.maketrans(" "," ", string.punctuation)
    line = line.translate(string.punctuation)
    line = line.translate(translation)
    test_x[index] = line

Here we can see that the text is lowered, having numbers and punctuations removed

In [15]:
index = 6
print(train_x[index])
print(test_x[index])

the wp xap works only on wp and the wp xap works only for wp
why not let us have several pages that we can put tiles on and name whatever we want to 


<h1>Designing the model</h1>

In [16]:
import tensorflow as tf
import numpy as np

In [17]:
# We need to import several things from Keras.
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU,LSTM, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

<h2>TOKENIZER</h2>

Since the model accepts only numerical values we need to translate the text. So we tokenize the text to arrays with numbers

In [18]:
num_words = 2000 # setting the max number of words = vocabulary
tokenizer = Tokenizer(num_words = num_words) # creating the tokenizer

In [19]:
data_text = train_x + test_x # creating our tokenizer with the entire dataset 

In [20]:
%%time 
tokenizer.fit_on_texts(data_text) # fitting the tokenizer to data

Wall time: 253 ms


Now that the tokenizer is ready, we convert the text to tokens using texts_to_sequences method in tokenizer

In [21]:
train_x_tokens = tokenizer.texts_to_sequences(train_x) 

This is the original text data

In [22]:
train_x[5]

'my app has a wp version and a wp version xap in the same submission'

This is the tokenized data

In [23]:
np.array(train_x_tokens[5])

array([ 31,  14,  64,   3,  81, 128,   4,   3,  81, 128, 802,   5,   1,
        91, 672])

Similarly for the test data

In [24]:
test_x_tokens = tokenizer.texts_to_sequences(test_x)

In [25]:
test_x[5]

'please add this simple and extremely helpful feature'

In [26]:
np.array(test_x_tokens[5])

array([ 46,  47,  13, 291,   4, 945, 624, 133])

Now it is needed to set the length of the input array. To find this we find the max length and the avg length of and compute the length of our input data

In [27]:
num_tokens = [len(tokens) for tokens in train_x_tokens + test_x_tokens]
num_tokens = np.array(num_tokens)

In [28]:
np.mean(num_tokens)

15.67410910690717

In [29]:
np.max(num_tokens)

193

In [30]:
# The max number of tokens is set to the average plus 2 standard deviations
max_tokens = np.mean(num_tokens)+ 2 * np.std(num_tokens)

#Converting the value to int
max_tokens = int(max_tokens)
print(max_tokens)

36


Now <b>pad the tokenized data</b>. Why? Cause differenet sentences have different lengths but our input length to the model is fixed so we need to pad the text to make it the same length as of the input

In [31]:
pad = 'pre' # add 0's to the first places of the array

In [32]:
train_x_pad = pad_sequences(train_x_tokens, maxlen = max_tokens, padding= pad, truncating = pad)

Here the sentences having length less than the input is padded

In [33]:
train_x_pad

array([[  33,    1,  205, ...,   43,    6,  472],
       [   0,  649,    5, ...,   16, 1534,    4],
       [   0,    0,    0, ...,   97, 1028,  557],
       ...,
       [   0,    0,    0, ..., 1757,    2,   39],
       [   0,    0,    0, ...,    3,  169,  633],
       [   0,    0,    0, ...,    8,  207,  148]])

Similarly we pad the test sequences

In [34]:
test_x_pad = pad_sequences(test_x_tokens, maxlen=max_tokens , padding = pad, truncating = pad)

In [35]:
# Tokenizer Inverse Map
#  a function that will translate the integer-tokens back to words

idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [36]:
# Helper function for converting a list of tokends back to a string of words

def tokens_to_string(tokens):
    # Map from tokens back to the words:
    words = [inverse_map[token] for token in tokens if token != 0]
    
    #Concatenate all the words
    text = " ".join(words)
    return text

In [37]:
train_x[5]

'my app has a wp version and a wp version xap in the same submission'

In [38]:
tokens_to_string(train_x_tokens[5])

'my app has a wp version and a wp version xap in the same submission'

<h1>Deep Learning Model</h1>
<br>
Here the first layer is <b>Embedding</b> layer which normalizes the tokens then a <b>LSTM</b> layer which returns a sequence to a <b>GRU</b> layer. Why? after experimenting with diffenent combinations this one found out to give best results

In [39]:
model = Sequential()

In [40]:
embedding_size = 8

In [41]:
model.add(Embedding(input_dim = num_words, output_dim = embedding_size
                    , input_length = max_tokens, name = 'layer_embedding'))

In [42]:
model.add(LSTM(8, return_sequences = True))

In [43]:
model.add(GRU(4))

In [44]:
model.add(Dense(1,activation='sigmoid'))

In [45]:
optimizer = Adam(lr=1e-3)
model.compile(loss ='binary_crossentropy', optimizer =  optimizer, metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 36, 8)             16000     
_________________________________________________________________
lstm (LSTM)                  (None, 36, 8)             544       
_________________________________________________________________
gru (GRU)                    (None, 4)                 156       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 16,705
Trainable params: 16,705
Non-trainable params: 0
_________________________________________________________________


Here we can see that the model gives an accuracy about 85%

In [66]:
%%time
model.fit(train_x_pad, train_y, validation_split = 0.05, epochs =7, batch_size = 64)

Train on 8075 samples, validate on 425 samples
Epoch 1/7
8075/8075 [==============================] - 3s 419us/step - loss: 0.2747 - acc: 0.8899 - val_loss: 0.1386 - val_acc: 0.9553
Epoch 2/7
8075/8075 [==============================] - 4s 435us/step - loss: 0.2499 - acc: 0.9029 - val_loss: 0.1541 - val_acc: 0.9529
Epoch 3/7
8075/8075 [==============================] - 4s 554us/step - loss: 0.2301 - acc: 0.9134 - val_loss: 0.1118 - val_acc: 0.9600
Epoch 4/7
8075/8075 [==============================] - 4s 528us/step - loss: 0.2163 - acc: 0.9207 - val_loss: 0.1260 - val_acc: 0.9529
Epoch 5/7
8075/8075 [==============================] - 5s 584us/step - loss: 0.2008 - acc: 0.9293 - val_loss: 0.1154 - val_acc: 0.9576
Epoch 6/7
8075/8075 [==============================] - 5s 605us/step - loss: 0.1897 - acc: 0.9337 - val_loss: 0.1272 - val_acc: 0.9553
Epoch 7/7
8075/8075 [==============================] - 5s 624us/step - loss: 0.1873 - acc: 0.9345 - val_loss: 0.1665 - val_acc: 0.9341
Wall tim

In [67]:
%%time
result = model.evaluate(test_x_pad, test_y)

592/592 [==============================] - 0s 174us/step
Wall time: 105 ms


Getting the accuracy on the test data

In [68]:
print("Accuracy : {0:.2%}".format(result[1]))

Accuracy : 76.18%


Now we give our own data to test our model

In [49]:
text1 = "Please add this simple and extremely useful feature "
text2 = "This is awesome"
text3 = "you should develop an app to count the time"

texts = [text1,text2,text3]

In [50]:
# Convert the text to integer tokens because that is what our machine demands
tokens = tokenizer.texts_to_sequences(texts)

In [51]:
# Padding out input so that it gonna be of the same length
tokens_pad = pad_sequences(tokens, maxlen=max_tokens, padding = pad, truncating= pad)

In [52]:
tokens_pad.shape

(3, 36)

Predicting the labels

In [53]:
model_out = model.predict(tokens_pad)

In [54]:
print(model_out)

[[0.8149213 ]
 [0.06187698]
 [0.6954872 ]]


<h1>Evaluation Data</h1>

In [55]:
eval_data = pd.read_csv('SubtaskA_EvaluationData.csv')

In [56]:
eval_data.head()

,9566,This would enable live traffic aware apps.,X
0,9569,Please try other formatting like bold italics ...,X
1,9576,Since computers were invented to save time I s...,X
2,9577,Allow rearranging if the user wants to change ...,X
3,9579,Add SIMD instructions for better use of ARM NE...,X
4,9580,Also using a hot swapping code generator (opti...,X


In [57]:
eval_data_text = eval_data.iloc[:,1]
eval_text = []
for text in eval_data_text:
    eval_text.append(text)


In [58]:
for index ,line in enumerate(eval_text):
    line = line.lower() #lowercase lines
    line = re.sub(r'\d+', '', line) #regular expression to remove the numbers
    translation = str.maketrans(" "," ", string.punctuation)
    line = line.translate(string.punctuation) #removes the punctuations
    line = line.translate(translation)
    train_x[index] = line # update the text files

In [59]:
tokenizer.fit_on_texts(eval_text)

In [60]:
eval_tokens = tokenizer.texts_to_sequences(eval_text) 

In [61]:
eval_pad = pad_sequences(eval_tokens, maxlen = max_tokens, padding= pad, truncating = pad)

In [62]:
evaluation = []
evaluation = model.predict(eval_pad)

In [63]:
for i, val in enumerate(evaluation):
    if val>0.5:
        evaluation[index] = 1
    else:
        evaluation[index] = 0


Creating dataframe to save the outputs and exporting to CSV

In [64]:
eval_submission = eval_data.iloc[:, [0,1]]
eval_output = pd.DataFrame(evaluation)
result = pd.concat([eval_submission,eval_output], axis=1, sort=False)

In [65]:
result.to_csv(r'Soumyajit_Rout.csv')

<h1>___________________________________________________</h1>